In [2]:
from layout_module import Layout, NestedTuple
from layout_module import layout_algebra as la

In [39]:
lane_shape = (4,4)
lane_stride = (1,4)
load_shape = (1,8)
load_stride = (0,1)
lane_layout = Layout(NestedTuple.from_literal(lane_shape), NestedTuple.from_literal(lane_stride))
load_layout = Layout(NestedTuple.from_literal(load_shape), NestedTuple.from_literal(load_stride))
loader = la.product(load_layout, lane_layout)

In [43]:
shared_memory_shape = NestedTuple.from_literal((16,8))
shared_memory_stride = NestedTuple.from_literal((8,1))
smem_layout = Layout(shared_memory_shape, shared_memory_stride)

access_layout = la.compose(smem_layout, loader)
access_layout.coalesce()

Layout(shape=(16, 8), stride=(8, 1))

In [45]:
warp_layout = Layout(NestedTuple.from_literal((32,2,2)), NestedTuple.from_literal((4,2,1)))